# Get data from The New York Times

In this notebook, we set up the functions to send and process multiple queries to the Article Search API provided by The New York Times. Each query asks for articles that contain a deaf-related phrase, such as "deaf and dumb" or "hearing-impaired". Full list of phrases below.

Contents:
- Define functions
- Get data
- Postprocessing

Import dependencies.

In [105]:
import os
import dateutil
import pandas as pd
#pd.options.display.max_colwidth = 100
import numpy as np
import requests
import time
from ast import literal_eval
import datetime
from dateutil.relativedelta import relativedelta

# Usernames and passwords
import configparser
configs = configparser.ConfigParser()
configs.read('../../config.ini')

['../../config.ini']

## Define functions

In [174]:
def send_request(query, page):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
    url = base_url + '?fq=' + query + '&api-key=' + configs['NYT']['ACCESS_KEY'] + '&page=' + str(page)
    response = requests.get(url).json()
    time.sleep(6)
    return response


def parse_response(response, data):
    '''Parses and returns response as pandas data frame.'''
    
    articles = response['response']['docs'] 
    for article in articles: 
        
        # id
        data['id'].append(article['_id'])
        
        # Date
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        
        # Headline
        data['headline'].append(article['headline']['main']) 
        
        # Section
        if 'section_name' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        
        # News desk
        if 'news_desk' in article:
            data['news_desk'].append(article['news_desk'])
        else:
            data['news_desk'].append(None)
        
        # Document type
        data['doc_type'].append(article['document_type'])
        
        # Type of material
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
            
        # Keywords
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
        
        # Web URL
        if 'web_url' in article:
            data['url'].append(article['web_url'])
            
        # Author
        if 'byline' in article:
            data['byline'].append(article['byline']['original'])
        else:
            data['byline'].append(None)
            
            
def send_query(query, data, date=None):
    # If the query has already been sent, don't send again
    if query + '.csv' in os.listdir('data'): 
        print('Already have data for the term "' + query + '".\n')
        return False
    
    # If date is provided, append to query string
    query_str = queries[query]
    if date:
        query_str = query_str + date
        
    print('Querying string: ' + query_str + '\n')
    
    page_num = 0
    while True:
        response = send_request(query_str, page_num)
        offset = response['response']['meta']['offset']
        hits = response['response']['meta']['hits']
        
        if offset > hits: 
            print('Done processing results.\n')
            return True
        # If we have 2,000 hits or more, we will need to break down our query into date intervals
        elif hits >= 2000: 
            print('We have over 2,000 hits.\n')
            # Send the same query again, once for each date interval
            for date in q_dates:
                send_query(query, data, date) 
            return True
            
        print('Processing results ' + str(offset) + '—' + str(min((offset + 10), hits)) + '/' + str(hits) + '...')
        parse_response(response, data)
        page_num += 1

## Get data

Our goal is to analyze the usage of deaf-related terms in *The New York Times*. 

The deaf-related terms: 

- "deaf and dumb"
- "deaf-mute"
- "hearing-impaired"
- "tone deaf"
- "deaf as a post"
- "stone deaf" 
- "fell on deaf ears"
- "deaf" (excluding its presence in all of the above terms) 

Some terms have varieties. For example, take "fell on deaf ears." Taking into consideration all of its varieties (that yielded hits when testing manually), the resulting query would be 

>`body:"fell on deaf ears" OR headline:"fell on deaf ears" 
OR body:"fall on deaf ears" OR headline:"fall on deaf ears" 
OR body:"falls on deaf ears" 
OR body:"fall on a deaf ear" 
OR body:"fell on a deaf ear" 
OR body:"turn a deaf ear" OR headline:"turn a deaf ear" 
OR body:"turned deaf ear" OR headline:"turned deaf ear"
OR body:"turned a deaf ear" OR headline:"turned a deaf ear"`

Construct remaining queries. I did this by checking the API manually in the browser, adding conditions one by one. 

http://api.nytimes.com/svc/search/v2/articlesearch.json?fq= + `query` + &api-key= + `api_key`

In [16]:
q_deaf_and_dumb = '''
body:"deaf and dumb" OR headline:"deaf and dumb"
OR body:"deaf dumb" OR headline:"deaf dumb"
'''.replace('\n', ' ').strip()

q_deaf_mute = '''
body:"deaf mute" OR headline:"deaf mute"
OR body:"deaf and mute" OR headline:"deaf and mute"
OR body:"mute deaf"
OR body:"mute and deaf" OR headline:"mute and deaf"
'''.replace('\n', ' ').strip()

q_fall_on_deaf_ears = '''
body:"fell on deaf ears" OR headline:"fell on deaf ears"
OR body:"fall on deaf ears" OR headline:"fall on deaf ears"
OR body:"falls on deaf ears"
OR body:"fall on a deaf ear"
OR body:"fell on a deaf ear"
OR body:"turn a deaf ear" OR headline:"turn a deaf ear"
OR body:"turned deaf ear" OR headline:"turned deaf ear"
OR body:"turned a deaf ear" OR headline:"turned a deaf ear"
'''.replace('\n', ' ').strip()

q_hearing_impaired = 'body:"hearing impaired" OR headline:"hearing impaired"'.replace('\n', ' ').strip()

q_tone_deaf = 'body:"tone deaf" OR headline:"tone deaf"'.replace('\n', ' ').strip()

q_deaf_as_a_post = 'body:"deaf as a post" OR headline:"deaf as a post"'.replace('\n', ' ').strip()

q_stone_deaf = 'body:"stone deaf" OR headline:"stone deaf"'.replace('\n', ' ').strip()

q_deaf = '(body: "deaf" OR headline:"deaf") AND NOT (' + q_deaf_and_dumb + ') AND NOT (' + q_deaf_mute + ') AND NOT (' + q_fall_on_deaf_ears + ') AND NOT (' + q_hearing_impaired + ') AND NOT (' + q_tone_deaf + ') AND NOT (' + q_deaf_as_a_post + ') AND NOT (' + q_stone_deaf + ')'\

queries = {'deaf_and_dumb': q_deaf_and_dumb,
           'deaf_mute': q_deaf_mute,
           'fall_on_deaf_ears': q_fall_on_deaf_ears,
           'hearing_impaired': q_hearing_impaired,
           'tone_deaf': q_tone_deaf,
           'deaf_as_a_post': q_deaf_as_a_post,
           'stone_deaf': q_stone_deaf,
           'deaf': q_deaf}

Date intervals for NYT search API are formatted as `&begin_date=20120101&end_date=20121231`. We need to format date intervals because if our query returns over 2,000 results, we can't get them all. We need to make our query smaller, and we can do that by narrowing the date interval. We will do 5-year intervals.

In [149]:
start, end = [datetime.datetime.strptime("1850-01-01", "%Y-%m-%d"), datetime.datetime.today()]
interval = 5 # years
dates = [(start + relativedelta(years=x)).strftime('%Y%m%d') for x in range(0, relativedelta(end, start).years + 10, interval)]
q_dates = ['&begin_date=' + start + '&end_date=' + end for start, end in zip(dates, dates[1:])]

Send these queries to the Archive API to create CSV tables that we save to `./data/`.

In [176]:
for query in list(queries.keys()):
    # Reset the global data object
    data = {'headline': [],  
            'date': [], 
            'doc_type': [],
            'material_type': [],
            'news_desk': [],
            'section': [],
            'keywords': [],
            'url': [],
            'id': [],
            'byline': []}
    
    # Send query
    got_result = send_query(query, data)
    
    # Build and save frame from data object 
    if got_result:
        data_df = pd.DataFrame(data)
        data_df['date'] = pd.to_datetime(data_df['date'])
        data_df.to_csv('data/' + query + '.csv', index=False)
        print('Saved as ' + query + '.csv.\n')

Querying string: body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb"

Processing results 0—10/727...
Processing results 10—20/727...
Processing results 20—30/727...
Processing results 30—40/727...
Processing results 40—50/727...
Processing results 50—60/727...
Processing results 60—70/727...
Processing results 70—80/727...
Processing results 80—90/727...
Processing results 90—100/727...
Processing results 100—110/727...
Processing results 110—120/727...
Processing results 120—130/727...
Processing results 130—140/727...
Processing results 140—150/727...
Processing results 150—160/727...
Processing results 160—170/727...
Processing results 170—180/727...
Processing results 180—190/727...
Processing results 190—200/727...
Processing results 200—210/727...
Processing results 210—220/727...
Processing results 220—230/727...
Processing results 230—240/727...
Processing results 240—250/727...
Processing results 250—260/727...
Processing results 260—27

Processing results 290—300/1669...
Processing results 300—310/1669...
Processing results 310—320/1669...
Processing results 320—330/1669...
Processing results 330—340/1669...
Processing results 340—350/1669...
Processing results 350—360/1669...
Processing results 360—370/1669...
Processing results 370—380/1669...
Processing results 380—390/1669...
Processing results 390—400/1669...
Processing results 400—410/1669...
Processing results 410—420/1669...
Processing results 420—430/1669...
Processing results 430—440/1669...
Processing results 440—450/1669...
Processing results 450—460/1669...
Processing results 460—470/1669...
Processing results 470—480/1669...
Processing results 480—490/1669...
Processing results 490—500/1669...
Processing results 500—510/1669...
Processing results 510—520/1669...
Processing results 520—530/1669...
Processing results 530—540/1669...
Processing results 540—550/1669...
Processing results 550—560/1669...
Processing results 560—570/1669...
Processing results 5

Processing results 900—910/1302...
Processing results 910—920/1302...
Processing results 920—930/1302...
Processing results 930—940/1302...
Processing results 940—950/1302...
Processing results 950—960/1302...
Processing results 960—970/1302...
Processing results 970—980/1302...
Processing results 980—990/1302...
Processing results 990—1000/1302...
Processing results 1000—1010/1302...
Processing results 1010—1020/1302...
Processing results 1020—1030/1302...
Processing results 1030—1040/1302...
Processing results 1040—1050/1302...
Processing results 1050—1060/1302...
Processing results 1060—1070/1302...
Processing results 1070—1080/1302...
Processing results 1080—1090/1302...
Processing results 1090—1100/1302...
Processing results 1100—1110/1302...
Processing results 1110—1120/1302...
Processing results 1120—1130/1302...
Processing results 1130—1140/1302...
Processing results 1140—1150/1302...
Processing results 1150—1160/1302...
Processing results 1160—1170/1302...
Processing results 1

Processing results 10—20/65...
Processing results 20—30/65...
Processing results 30—40/65...
Processing results 40—50/65...
Processing results 50—60/65...
Processing results 60—65/65...
Done processing results.

Saved as stone_deaf.csv.

Querying string: (body: "deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR body:"fall on a deaf ear" OR body:"fell on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ear" OR headline:"turned deaf ear" OR body:"turned a deaf ear" OR headline:"turned a deaf ear") AND NOT (body:"hearing impaired" OR headline:"h

Processing results 0—10/10...
Processing results 10—10/10...
Done processing results.

Querying string: (body: "deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR body:"fall on a deaf ear" OR body:"fell on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ear" OR headline:"turned deaf ear" OR body:"turned a deaf ear" OR headline:"turned a deaf ear") AND NOT (body:"hearing impaired" OR headline:"hearing impaired") AND NOT (body:"tone deaf" OR headline:"tone deaf") AND NOT (body:"deaf as a post" OR headline:"deaf as a post") AND NOT (body:"stone 

...

Processing results 430—440/779...
Processing results 440—450/779...
Processing results 450—460/779...
Processing results 460—470/779...
Processing results 470—480/779...
Processing results 480—490/779...
Processing results 490—500/779...
Processing results 500—510/779...
Processing results 510—520/779...
Processing results 520—530/779...
Processing results 530—540/779...
Processing results 540—550/779...
Processing results 550—560/779...
Processing results 560—570/779...
Processing results 570—580/779...
Processing results 580—590/779...
Processing results 590—600/779...
Processing results 600—610/779...
Processing results 610—620/779...
Processing results 620—630/779...
Processing results 630—640/779...
Processing results 640—650/779...
Processing results 650—660/779...
Processing results 660—670/779...
Processing results 670—680/779...
Processing results 680—690/779...
Processing results 690—700/779...
Processing results 700—710/779...
Processing results 710—720/779...
Processing res

Processing results 440—450/691...
Processing results 450—460/691...
Processing results 460—470/691...
Processing results 470—480/691...
Processing results 480—490/691...
Processing results 490—500/691...
Processing results 500—510/691...
Processing results 510—520/691...
Processing results 520—530/691...
Processing results 530—540/691...
Processing results 540—550/691...
Processing results 550—560/691...
Processing results 560—570/691...
Processing results 570—580/691...
Processing results 580—590/691...
Processing results 590—600/691...
Processing results 600—610/691...
Processing results 610—620/691...
Processing results 620—630/691...
Processing results 630—640/691...
Processing results 640—650/691...
Processing results 650—660/691...
Processing results 660—670/691...
Processing results 670—680/691...
Processing results 680—690/691...
Processing results 690—691/691...
Done processing results.

Querying string: (body: "deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headli

Processing results 700—710/855...
Processing results 710—720/855...
Processing results 720—730/855...
Processing results 730—740/855...
Processing results 740—750/855...
Processing results 750—760/855...
Processing results 760—770/855...
Processing results 770—780/855...
Processing results 780—790/855...
Processing results 790—800/855...
Processing results 800—810/855...
Processing results 810—820/855...
Processing results 820—830/855...
Processing results 830—840/855...
Processing results 840—850/855...
Processing results 850—855/855...
Done processing results.

Querying string: (body: "deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR head

Processing results 670—680/1063...
Processing results 680—690/1063...
Processing results 690—700/1063...
Processing results 700—710/1063...
Processing results 710—720/1063...
Processing results 720—730/1063...
Processing results 730—740/1063...
Processing results 740—750/1063...
Processing results 750—760/1063...
Processing results 760—770/1063...
Processing results 770—780/1063...
Processing results 780—790/1063...
Processing results 790—800/1063...
Processing results 800—810/1063...
Processing results 810—820/1063...
Processing results 820—830/1063...
Processing results 830—840/1063...
Processing results 840—850/1063...
Processing results 850—860/1063...
Processing results 860—870/1063...
Processing results 870—880/1063...
Processing results 880—890/1063...
Processing results 890—900/1063...
Processing results 900—910/1063...
Processing results 910—920/1063...
Processing results 920—930/1063...
Processing results 930—940/1063...
Processing results 940—950/1063...
Processing results 9

Processing results 150—160/857...
Processing results 160—170/857...
Processing results 170—180/857...
Processing results 180—190/857...
Processing results 190—200/857...
Processing results 200—210/857...
Processing results 210—220/857...
Processing results 220—230/857...
Processing results 230—240/857...
Processing results 240—250/857...
Processing results 250—260/857...
Processing results 260—270/857...
Processing results 270—280/857...
Processing results 280—290/857...
Processing results 290—300/857...
Processing results 300—310/857...
Processing results 310—320/857...
Processing results 320—330/857...
Processing results 330—340/857...
Processing results 340—350/857...
Processing results 350—360/857...
Processing results 360—370/857...
Processing results 370—380/857...
Processing results 380—390/857...
Processing results 390—400/857...
Processing results 400—410/857...
Processing results 410—420/857...
Processing results 420—430/857...
Processing results 430—440/857...
Processing res

Merge all dataframes into one, with True or False for whether they contain a particular phrase. This means we're setting these additional `bool` columns:

- deaf_and_dumb
- deaf_mute
- fall_on_deaf_ears
- hearing_impaired
- tone_deaf
- deaf_as_a_post
- stone_deaf
- deaf

In [745]:
final_df = pd.DataFrame(columns=data.keys())

# For each key
for key in queries.keys():
    # Read in its CSV
    df = pd.read_csv('data/' + key + '.csv')
    
    # Drop any duplicates for this term alone
    num_dupes = len(df[df['id'].duplicated()])
    if num_dupes > 0:
        print('Dropping duplicates: ' + str(num_dupes))
        df = df.drop_duplicates(subset='id')
    print('added to final_df the df for key ' + key + ' with ' + str(len(df)) + ' values')
    
    df[key] = True # Label each row of this df as belonging to that df
    final_df = pd.concat([final_df, df], axis=0) # Add the df to the final df
    final_df.reset_index(drop=True, inplace=True)

print('\nTotal values: ' + str(len(final_df)))

added to final_df the df for key deaf_and_dumb with 727 values
added to final_df the df for key deaf_mute with 1120 values
added to final_df the df for key fall_on_deaf_ears with 1669 values
added to final_df the df for key hearing_impaired with 1302 values
added to final_df the df for key tone_deaf with 1739 values
added to final_df the df for key deaf_as_a_post with 22 values
added to final_df the df for key stone_deaf with 65 values
Dropping duplicates: 8
added to final_df the df for key deaf with 10905 values

Total values: 17549


In [746]:
final_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17544,"Alexa, Awake",2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Science and Technology']",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/c104e18e-f797-5490-b03c-259b...,By Brian Turner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
17545,Confessions of a Dating Profile,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Artificial Intelligence', 'Dating and Relati...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/ccaba5b4-672d-568b-9daa-b6bb...,By Eric Kaplan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
17546,How to Make Billions in E-Sports,2020-02-19,multimedia,Interactive Feature,Magazine,Magazine,"['E-Sports', 'Computer and Video Games', 'Fash...",https://www.nytimes.com/interactive/2020/02/18...,nyt://interactive/dd0dfee2-c9c6-5f58-acde-0801...,By Robert Capps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
17547,Parent-Teacher Association,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Education (K-12)', 'Children and ...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/df6cd908-52b0-5851-ac81-e7b8...,By Jessica Powell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


## Postprocessing

Go back through each key's column and set `NaN` to `False`. 

In [747]:
for key in queries.keys():
    final_df[key].fillna(False, inplace=True)
final_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,False,False,False,False,False,False,False
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,False,False,False,False,False,False,False
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,False,False,False,False,False,False,False
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,False,False,False,False,False,False,False
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17544,"Alexa, Awake",2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Science and Technology']",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/c104e18e-f797-5490-b03c-259b...,By Brian Turner,False,False,False,False,False,False,False,True
17545,Confessions of a Dating Profile,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Artificial Intelligence', 'Dating and Relati...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/ccaba5b4-672d-568b-9daa-b6bb...,By Eric Kaplan,False,False,False,False,False,False,False,True
17546,How to Make Billions in E-Sports,2020-02-19,multimedia,Interactive Feature,Magazine,Magazine,"['E-Sports', 'Computer and Video Games', 'Fash...",https://www.nytimes.com/interactive/2020/02/18...,nyt://interactive/dd0dfee2-c9c6-5f58-acde-0801...,By Robert Capps,False,False,False,False,False,False,False,True
17547,Parent-Teacher Association,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Education (K-12)', 'Children and ...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/df6cd908-52b0-5851-ac81-e7b8...,By Jessica Powell,False,False,False,False,False,False,False,True


Some articles contain multiple phrases. For example an article may contain "deaf-mute" as well as "hearing-impaired." They currently show up as two separate rows, with duplicate values for everything except the deaf-mute and hearing-impaired columns. One row shows True in the deaf-mute column, and the other row shows True in the hearing-impaired column. We want to merge these two rows into one row where both columns show True.

In [769]:
if final_df['id'].duplicated().any():
    print('Duplicates: ' + str(final_df['id'].duplicated().sum()))

Duplicates: 74


I tried to process the duplicates in the data frame like this:

In [770]:
final_df.groupby(df.columns[0:10].tolist(), as_index=False).max() # DOESNT WORK, DROPS TOO MANY ROWS

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,'A Little Bit of Help',1985-09-07,article,News,Metropolitan Desk,New York,"['Culture', 'Awards, Decorations and Honors', ...",https://www.nytimes.com/1985/09/07/nyregion/ne...,nyt://article/d9a9b191-341c-5ca2-975e-7c26d0c3...,By Susan Heller Anderson and David W. Dunlap,False,False,False,False,False,False,False,True
1,"'Beach House,' romantic comedy at Circle Rep.",1985-03-29,article,News,Weekend Desk,Theater,['Theater'],https://www.nytimes.com/1985/03/29/theater/bro...,nyt://article/1d7824c8-8ec3-5af2-809b-f11b56b3...,By Enid Nemy,False,False,False,True,False,False,False,False
2,"'Noises Off,' With Flawless Timing",1990-07-08,article,News,Westchester Weekly Desk,New York,"['Reviews', 'Theater']",https://www.nytimes.com/1990/07/08/nyregion/th...,nyt://article/6f7f506a-308a-5679-bc58-79cfa26f...,By Alvin Klein,False,False,False,False,False,False,False,True
3,10TH ANNIVERSARY FOR 'SUMMER EVENINGS',1985-06-09,article,Review,Westchester Weekly Desk,New York,['Music'],https://www.nytimes.com/1985/06/09/nyregion/mu...,nyt://article/801429bd-4a71-5285-8a61-bf2ba672...,By Robert Sherman,False,False,False,False,False,False,False,True
4,A 'STREAMLINED' EDITION OF 1985 OSCAR AWARDS,1985-03-27,article,Review,Cultural Desk,Movies,"['MOTION PICTURES', 'Television']",https://www.nytimes.com/1985/03/27/movies/tv-r...,nyt://article/a332e0ad-1165-5bf7-891c-1f1670ea...,By Janet Maslin,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11951,‘Wonder Woman’ Could Be the Superhero Women in...,2017-06-04,article,News,Business,Business Day,"['Movies', 'Women and Girls', 'Sexual Harassme...",https://www.nytimes.com/2017/06/04/business/me...,nyt://article/234b9c6e-59ed-5dfb-a561-2939c35c...,By Jim Rutenberg,False,False,False,True,False,False,False,False
11952,‘World Wide Mind’,2011-02-15,article,Text,Science,Science,"['Books and Literature', 'Science and Technolo...",https://www.nytimes.com/2011/02/15/science/15s...,nyt://article/9c18470b-2b77-55bf-863a-fb55bef5...,By Michael Chorost,False,False,False,False,False,False,False,True
11953,‘Write When You Get Work’ Review: Backstreet B...,2018-11-22,article,Review,Weekend,Movies,['Movies'],https://www.nytimes.com/2018/11/22/movies/writ...,nyt://article/db879a75-4d65-56ff-b5ad-93ecf008...,By Jeannette Catsoulis,False,False,False,False,True,False,False,False
11954,"‘Yellow Vests’ Riot in Paris, but Their Anger ...",2018-12-02,article,News,Foreign,World,"['Yellow Vests Movement', 'Demonstrations, Pro...",https://www.nytimes.com/2018/12/02/world/europ...,nyt://article/4f02b33e-94f3-5d46-9fa9-71e3c50a...,By Adam Nossiter,False,False,False,False,False,False,False,True


But it didn't work. So I copied duplicates into a separate dataframe, processed them there, removed all copies of them from final_df, then appended the processed dupes to final_df and resorted. *shrug*

In [767]:
final_df.groupby(final_df.columns[0:10].tolist(), as_index=False).max()

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,'A Little Bit of Help',1985-09-07,article,News,Metropolitan Desk,New York,"['Culture', 'Awards, Decorations and Honors', ...",https://www.nytimes.com/1985/09/07/nyregion/ne...,nyt://article/d9a9b191-341c-5ca2-975e-7c26d0c3...,By Susan Heller Anderson and David W. Dunlap,False,False,False,False,False,False,False,True
1,"'Beach House,' romantic comedy at Circle Rep.",1985-03-29,article,News,Weekend Desk,Theater,['Theater'],https://www.nytimes.com/1985/03/29/theater/bro...,nyt://article/1d7824c8-8ec3-5af2-809b-f11b56b3...,By Enid Nemy,False,False,False,True,False,False,False,False
2,"'Noises Off,' With Flawless Timing",1990-07-08,article,News,Westchester Weekly Desk,New York,"['Reviews', 'Theater']",https://www.nytimes.com/1990/07/08/nyregion/th...,nyt://article/6f7f506a-308a-5679-bc58-79cfa26f...,By Alvin Klein,False,False,False,False,False,False,False,True
3,10TH ANNIVERSARY FOR 'SUMMER EVENINGS',1985-06-09,article,Review,Westchester Weekly Desk,New York,['Music'],https://www.nytimes.com/1985/06/09/nyregion/mu...,nyt://article/801429bd-4a71-5285-8a61-bf2ba672...,By Robert Sherman,False,False,False,False,False,False,False,True
4,A 'STREAMLINED' EDITION OF 1985 OSCAR AWARDS,1985-03-27,article,Review,Cultural Desk,Movies,"['MOTION PICTURES', 'Television']",https://www.nytimes.com/1985/03/27/movies/tv-r...,nyt://article/a332e0ad-1165-5bf7-891c-1f1670ea...,By Janet Maslin,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11951,‘Wonder Woman’ Could Be the Superhero Women in...,2017-06-04,article,News,Business,Business Day,"['Movies', 'Women and Girls', 'Sexual Harassme...",https://www.nytimes.com/2017/06/04/business/me...,nyt://article/234b9c6e-59ed-5dfb-a561-2939c35c...,By Jim Rutenberg,False,False,False,True,False,False,False,False
11952,‘World Wide Mind’,2011-02-15,article,Text,Science,Science,"['Books and Literature', 'Science and Technolo...",https://www.nytimes.com/2011/02/15/science/15s...,nyt://article/9c18470b-2b77-55bf-863a-fb55bef5...,By Michael Chorost,False,False,False,False,False,False,False,True
11953,‘Write When You Get Work’ Review: Backstreet B...,2018-11-22,article,Review,Weekend,Movies,['Movies'],https://www.nytimes.com/2018/11/22/movies/writ...,nyt://article/db879a75-4d65-56ff-b5ad-93ecf008...,By Jeannette Catsoulis,False,False,False,False,True,False,False,False
11954,"‘Yellow Vests’ Riot in Paris, but Their Anger ...",2018-12-02,article,News,Foreign,World,"['Yellow Vests Movement', 'Demonstrations, Pro...",https://www.nytimes.com/2018/12/02/world/europ...,nyt://article/4f02b33e-94f3-5d46-9fa9-71e3c50a...,By Adam Nossiter,False,False,False,False,False,False,False,True


In [771]:
# Process dupes separately
dupes = pd.concat(g for _, g in final_df.groupby('id') if len(g) > 1)
dupes

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
10,MISCELLANEOUS.; A NOVEL SHIP-OF-WAR.,1864-10-09,article,Archives,None,Archives,['FOREIGN'],https://www.nytimes.com/1864/10/09/archives/mi...,nyt://article/03b9f824-9944-526a-9e88-cd15ecf6...,From the London Times,True,False,False,False,False,False,False,False
741,MISCELLANEOUS.; A NOVEL SHIP-OF-WAR.,1864-10-09,article,Archives,None,Archives,['FOREIGN'],https://www.nytimes.com/1864/10/09/archives/mi...,nyt://article/03b9f824-9944-526a-9e88-cd15ecf6...,From the London Times,False,True,False,False,False,False,False,False
11,School for Deaf Mutes.,1861-04-04,article,Archives,None,Archives,[],https://www.nytimes.com/1861/04/04/archives/sc...,nyt://article/03d8bd5b-cb34-5469-af5e-0d57e7e9...,NaN,True,False,False,False,False,False,False,False
743,School for Deaf Mutes.,1861-04-04,article,Archives,None,Archives,[],https://www.nytimes.com/1861/04/04/archives/sc...,nyt://article/03d8bd5b-cb34-5469-af5e-0d57e7e9...,NaN,False,True,False,False,False,False,False,False
13,THE VIEW FROM: THE AMERICAN SCHOOL FOR THE DEA...,1988-04-17,article,News,Connecticut Weekly Desk,New York,['Deafness'],https://www.nytimes.com/1988/04/17/nyregion/th...,nyt://article/0436b734-ab2e-5c0b-8032-fefaeee0...,By Jacqueline Weaver,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,Health,1979-05-20,article,Archives,None,Archives,"['Theater', 'Repertory and Resident Theaters',...",https://www.nytimes.com/1979/05/20/archives/he...,nyt://article/f9c5ea03-b682-5860-8e40-fb976237...,By Patricia Curtis,False,False,False,True,False,False,False,False
1823,"In Cambodia, giving a stage to 'inclusive arts'",2008-06-24,article,News,IHT News,Arts,[],https://www.nytimes.com/2008/06/25/arts/25iht-...,nyt://article/fbba69d5-0ddf-518b-9961-39e9c551...,By Robert Turnbull,False,True,False,False,False,False,False,False
4791,"In Cambodia, giving a stage to 'inclusive arts'",2008-06-24,article,News,IHT News,Arts,[],https://www.nytimes.com/2008/06/25/arts/25iht-...,nyt://article/fbba69d5-0ddf-518b-9961-39e9c551...,By Robert Turnbull,False,False,False,True,False,False,False,False
722,OUTSIDER IN A SILENT WORLD,1986-08-31,article,News,Magazine Desk,Magazine,"['Deafness', 'Families and Family Life']",https://www.nytimes.com/1986/08/31/magazine/ou...,nyt://article/fe0fcaa7-9a86-56cb-bd20-e8f9394e...,BY Lou Ann Walker,True,False,False,False,False,False,False,False


Merge rows so that there is only one row for each unique id, retaining all True values. Note again that for each unique id, the only thing that differs are the columns containing the True/False values, hence us doing groupby on all the other columns so that we can apply .max() to those True/False columns.

.max() works cuz True > False

In [773]:
dupes.reset_index(drop=True, inplace=True)
dupes_fixed = dupes.groupby(dupes.columns[0:10].tolist(), as_index=False).max()
dupes_fixed

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,A Born Deadeye Gets a Job As (What Else?) a Hi...,2001-11-23,article,Review,"Movies, Performing Arts/Weekend Desk",Movies,"['MOTION PICTURES', 'Reviews']",https://www.nytimes.com/2001/11/23/movies/film...,nyt://article/50c8fe45-a573-5f74-ad04-709442ff...,By Dave Kehr,False,True,False,True,False,False,False,False
1,A Deaf Folk Artist Who Keenly Saw the World Ar...,2006-08-25,article,News,Leisure/Weekend Desk,Books,"['Deafness', 'Books and Literature', 'Museums'...",https://www.nytimes.com/2006/08/25/arts/a-deaf...,nyt://article/dcdb7013-b330-534e-8176-dbac57f9...,By Wendy Moonan,True,True,False,False,False,False,False,False
2,A Deaf Folk Artist Who Keenly Saw the World Ar...,2006-08-25,article,News,Weekend,Arts,"['Art', 'Museums', 'Deafness']",https://www.nytimes.com/2006/08/25/arts/design...,nyt://article/15d83131-0995-517c-9c67-85590f58...,By Wendy Moonan,True,True,False,False,False,False,False,False
3,A Misdiagnosed Deaf Man's Ordeal,1988-12-11,article,News,Metropolitan Desk,New York,"['Deafness', 'Malpractice', 'SUITS AND CLAIMS ...",https://www.nytimes.com/1988/12/11/nyregion/a-...,nyt://article/696b9fda-8a13-5383-8579-da9dec9c...,Special to the New York Times,False,True,False,True,False,False,False,False
4,A Nope for Pope,2010-03-27,article,Op-Ed,Editorial,Opinion,"['Nuns', 'Sex Crimes', 'Celibacy', 'Priests', ...",https://www.nytimes.com/2010/03/28/opinion/28d...,nyt://article/13d67aeb-1542-508b-b2c4-6a693f37...,By Maureen Dowd,True,False,False,False,True,False,False,False
5,A Pro and a Con,1978-04-15,article,Archives,None,Archives,[],https://www.nytimes.com/1978/04/15/archives/a-...,nyt://article/d1ecb8c4-25d3-54ad-b682-a10dc958...,By William O'Rourke,False,True,False,False,False,False,True,False
6,A Storm Over New Uses for Oral School at Mystic,1979-10-21,article,Archives,None,Archives,[],https://www.nytimes.com/1979/10/21/archives/co...,nyt://article/85bbfa73-f8d4-52af-96cb-2f277f5b...,By Steven Slosberg,True,False,False,True,False,False,False,False
7,Aiming at China’s Armpits: When Foreign Brands...,2018-02-03,article,News,Business,Business Day,"['Deodorants', 'Consumer Behavior', 'ADVERTISI...",https://www.nytimes.com/2018/02/02/business/ch...,nyt://article/1814e6d9-ecd4-5aa3-924e-c9b97bd6...,By Owen Guo,False,False,True,False,True,False,False,False
8,"BRIEF ENCOUNTER, AFTER HALF A CENTURY",1982-03-21,article,News,New Jersey Weekly Desk,New York,['Families and Family Life'],https://www.nytimes.com/1982/03/21/nyregion/br...,nyt://article/5bf411bc-5e09-5e02-8cef-3792229a...,By Joan Lowell Smith,True,True,False,False,False,False,False,False
9,Base Ball.,1865-11-13,article,Archives,None,Archives,['MISCELLANEOUS'],https://www.nytimes.com/1865/11/13/archives/ba...,nyt://article/ddd5df94-da98-5936-87a2-3b55f149...,FANWOOD VS. ALERT,True,True,False,False,False,False,False,False


Remove all instances of duplicates from final_df.

In [787]:
final_df = final_df[~final_df['id'].isin(dupes['id'])]
final_final_lol_mybad_df = pd.concat((final_df, dupes_fixed))
final_final_lol_mybad_df.reset_index(inplace=True, drop=True)
final_final_lol_mybad_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fall_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,False,False,False,False,False,False,False
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,False,False,False,False,False,False,False
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,False,False,False,False,False,False,False
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,False,False,False,False,False,False,False
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17442,Theater: ‘Look to the Lilies’ Begins Its Run a...,1970-03-30,article,Archives,None,Archives,"['Theater', 'REVIEWS AND OTHER DATA ON SPECIFI...",https://www.nytimes.com/1970/03/30/archives/th...,nyt://article/be014601-0c8f-5182-9bf1-fe762c9f...,By Clive Barnes,False,False,False,False,True,False,True,False
17443,WESTCHESTER Q&A;: MARIE TRAFICANTE;\nBringing ...,1993-01-10,article,Interview,Westchester Weekly Desk,New York,"['Music', 'Teachers and School Employees']",https://www.nytimes.com/1993/01/10/nyregion/we...,nyt://article/12beaf0c-59ab-52e7-8449-4a20ab2c...,By Donna Greene,False,False,False,True,True,False,False,False
17444,WESTCHESTER Q&A;: MARIE TRAFICANTE;\nBringing ...,1993-01-10,article,Interview,Westchester Weekly Desk,New York,"['Music', 'Teachers and School Employees']",https://www.nytimes.com/1993/01/10/nyregion/we...,nyt://article/cc68f54b-25d0-5fb5-91aa-9c225d27...,By Donna Greene,False,False,False,True,True,False,False,False
17445,WESTCHESTER Q&A;: MARIE tRAFICANTE;\nBringing ...,1993-01-10,article,Interview,Westchester Weekly Desk,New York,"['Music', 'Teachers and School Employees']",https://www.nytimes.com/1993/01/10/nyregion/we...,nyt://article/13570b93-5f1a-5134-8b64-952ed3c6...,By Donna Greene,False,False,False,True,True,False,False,False


In [791]:
print('Duplicates: ' + str(final_final_lol_mybad_df['id'].duplicated().sum()))

Duplicates: 0


In [792]:
final_final_lol_mybad_df.to_csv('data/all.csv', index=False)